In [13]:
import os
import json
import openai
from openai import OpenAI
from dotenv import find_dotenv, load_dotenv
from uuid import uuid4
from random import randint
from utils.get_openai_api_key import get_test_key
from utils.get_postgres_connection import _conn_open
from utils.load_json import load_json
from rich.console import Console

console = Console()

In [14]:
OPENAI_API_KEY = get_test_key()
# Set up your OpenAI API key
openai.api_key = OPENAI_API_KEY
client = OpenAI()
# Choose a model
model = "text-embedding-ada-002"
console.print(OPENAI_API_KEY)

Success: .env file found with some environment variables

sk-proj-VN9eqRjnzkrZj83pElo5sx852vSchq-DUjCCWaQro9c3n5B2trKFzY8PrR45ViMlHkJV3SU265T3BlbkFJ3XOY3mCXNvvgfhPGFKkiIrSUR
HLarTdW0amlPwHXi0btb1fLdM9XVywYa8kJNneYQwY7l5yRQA

In [15]:
def get_embedding(text, model="text-embedding-3-small"):

    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

In [16]:
conn = _conn_open()
cur = conn.cursor()

Conn: <connection object at 0x000002670F411030; dsn: 'user=postgres password=xxx dbname=postgres 
host=host.docker.internal', closed: 0>

In [17]:
console = Console()
FILENAME = "test_article_elements.json"
json_data = load_json(FILENAME)
print(f"Number of items: {len(json_data)}")
encoded_image_data = None
for i in range(len(json_data)):

    if json_data[i]["type"] == "Image":
        console.print(f"[dark_orange]---------Item Number IMAGE: {i + 1}--------[/]")
        el_image_b64 = json_data[i]["metadata"]["image_base64"]

        # console.print(el_image_b64[:40])
        new_id = str(uuid4())
        new_element_id = randint(1000, 9999)
        sql = f"""
        INSERT INTO tbl_doc_elements (
            file_id, element_id, element_type, image_base64)
        VALUES 
        ('{new_id}','{new_element_id}','IMAGE','{el_image_b64}')
        RETURNING id;
        """
        # console.print(sql)
        try:
            cur.execute(sql)
            row = cur.fetchone()
            id = row[0]
            console.print(f"ID is {id}")
            conn.commit()
        except Exception as e:
            console.print(e)
    if json_data[i]["type"] == "NarrativeText":

        console.print(f"[dark_orange]---------Item Number TEXT: {i + 1}--------[/]")
        el_text = json_data[i]["text"]
        el_embedding = get_embedding(el_text)

        # console.print(el_embedding)
        new_id = str(uuid4())
        new_element_id = randint(1000, 9999)
        sql = f"""
        INSERT INTO tbl_doc_elements (
            file_id, element_id, element_text, element_type, image_base64, embedding)
        VALUES 
        ('{new_id}','{new_element_id}','{el_text}','NARRATIVE_TEXT',
        '{encoded_image_data}', '{el_embedding}')
        RETURNING id;
        """

        try:
            cur.execute(sql)
            row = cur.fetchone()
            id = row[0]
            console.print(f"ID is {id}")
            conn.commit()
        except Exception as e:
            console.print(e)

Number of items: 12


---------Item Number IMAGE: 1--------

ID is 1c968780-9eb4-4bbf-b3dc-6fbdeaed82e4

---------Item Number TEXT: 2--------

ID is b8f45ae7-8dfb-47e3-8cf0-ae61e95c1d4b

...

ID is 793eae2c-bec8-4eca-952d-6906b2920393

---------Item Number TEXT: 11--------

ID is 9d51dffd-d952-4ed7-bb55-aeae898ea94c

---------Item Number TEXT: 12--------

ID is 76e8fafe-cc5a-4270-bab5-e3faf6633703

In [18]:
cur.close()
conn.close()